<a href="https://colab.research.google.com/github/Baraa710/Premier_League_Predictor/blob/main/DataScrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: code to use firefox webdriver in google colab

!apt-get update
!apt install firefox-esr
!wget https://github.com/mozilla/geckodriver/releases/download/v0.34.0/geckodriver-v0.34.0-linux64.tar.gz
!tar -xzvf geckodriver-v0.34.0-linux64.tar.gz
!mv geckodriver /usr/local/bin
!pip install selenium



Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,081 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,357 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://ppa.launchpadcontent.net/deadsna

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import os
import multiprocessing
import requests

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
ranges = [(n, n+1) for n in range(2006, 2009)]

seasons = ["{}-{}".format(year1,year2) for (year1, year2) in ranges]
stat_types = [('playingtime','stats_playing_time')]

In [ ]:
!pip install xlsxwriter
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.1 MB/s eta 0:00:00


In [ ]:
def get_database(season, stat_type, table_id):
  print("starting to get season {}".format(season))
  driver.get("https://fbref.com/en/comps/9/{}/{}/{}-Premier-League-Stats".format(season,stat_type, season))
  # Wait for the page to load dynamically
  driver.implicitly_wait(5)

  # Get the page source
  html_data = driver.page_source

  # Parse the HTML data using BeautifulSoup
  soup = BeautifulSoup(html_data, "html.parser")
  if not soup:
    print("No soup found")
    return
  # Find the table with id "stats_standard"
  table = soup.find('table', {'id': table_id})
  if table == None:
    print("Could not find table for season {} stat type {}".format(season, stat_type))
    return
  print("found table\n")
  # Extract all rows from the table
  rows = table.find_all('tr')
  col_names = rows[1].find_all('th')
  col_names = [ele.text.strip() for ele in col_names]
  # Create an empty list to store the data
  data = []

  # Loop through the rows and extract the data from each row, skipping the first row
  for row in rows[2:]:
      cols = row.find_all('th') + row.find_all('td')
      cols = [ele.text.strip() for ele in cols]
      if(cols[0].isdigit()!=True):
        continue
      data.append(cols)  # Get rid of empty values

  # Create a DataFrame from the data list
  df = pd.DataFrame(data, columns = col_names)
  filename = '/content/gdrive/MyDrive/APS360_Project/Player_data/{}.xlsx'.format(season)
  if not os.path.isfile(filename):
    # Create a new file
    with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
        df.to_excel(writer, sheet_name=table_id)
  else:
    with pd.ExcelWriter(filename, mode='a', if_sheet_exists='new', engine='openpyxl') as writer:
      df.to_excel(writer, sheet_name=table_id)
  print("finished processing season {} stat {}".format(season, stat_type))




def process_season(season):
  print("processing season {}".format(season))
  for i,(stat_type, table_id) in enumerate(stat_types):
    print("processing season {} stat {}".format(season, stat_type))
    get_database(season, stat_type, table_id)

# Initialize driver
options = webdriver.FirefoxOptions()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
# Disable images
options.set_preference("permissions.default.image", 2)

driver = webdriver.Firefox(options=options)

# for season in seasons:
#   process_season(season)

# driver.quit()

In [ ]:
process_season('2022-2023')

processing season 2022-2023
processing season 2022-2023 stat playingtime
starting to get season 2022-2023
found table

finished processing season 2022-2023 stat playingtime


In [ ]:
ranges = [(n, n+1) for n in range(2006, 2023)]
seasons = ["{}-{}".format(year1,year2) for (year1, year2) in ranges]
seasons

['2006-2007',
 '2007-2008',
 '2008-2009',
 '2009-2010',
 '2010-2011',
 '2011-2012',
 '2012-2013',
 '2013-2014',
 '2014-2015',
 '2015-2016',
 '2016-2017',
 '2017-2018',
 '2018-2019',
 '2019-2020',
 '2020-2021',
 '2021-2022',
 '2022-2023']

In [ ]:
def get_squad_data(season, stat_type):
# Define the URL to scrape
  url = "https://fbref.com/en/comps/10/{}/{}/{}-Championship-Stats".format(season,stat_type, season)

  # Fetch the HTML content
  response = requests.get(url)
  html_content = response.content
  # Parse the HTML content
  soup = BeautifulSoup(html_content, "html.parser")
  if not soup:
    print("No soup found")
  # Extract the first two tables
  tables = soup.find_all('table')

  if not tables:
    print("No table found for season {}".format(season))
    return
  # Create a Pandas DataFrame for each table
  dfs = [pd.read_html(str(table))[0] for table in tables]
  filename = '/content/gdrive/MyDrive/APS360_Project/Data_2/{}.xlsx'.format(season)
  # write to excel
  if not os.path.isfile(filename):
    # Create a new file
    with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
        dfs[0].to_excel(writer, sheet_name=stat_type)
  with pd.ExcelWriter(filename, mode='a', if_sheet_exists='replace', engine='openpyxl') as writer:
          dfs[0].to_excel(writer, sheet_name='EFL Squad Data {}'.format(stat_type))

def process_season(season):
  print("processing season {}".format(season))
  for i,(stat_type, table_id) in enumerate(stat_types):
    print("processing season {} stat {}".format(season, stat_type))
    get_squad_data(season, stat_type)

num_processes = multiprocessing.cpu_count()
pool = multiprocessing.Pool(processes=num_processes)
pool.map(process_season, seasons)
pool.close()
pool.join()

processing season 2009-2010
processing season 2006-2007processing season 2009-2010 stat playingtime
processing season 2006-2007 stat playingtime

processing season 2010-2011
processing season 2010-2011 stat playingtime
processing season 2007-2008
processing season 2007-2008 stat playingtime
processing season 2011-2012
processing season 2011-2012 stat playingtime
processing season 2008-2009
processing season 2008-2009 stat playingtime
processing season 2012-2013
processing season 2012-2013 stat playingtime
processing season 2015-2016
processing season 2015-2016 stat playingtime
processing season 2013-2014
processing season 2013-2014 stat playingtime
processing season 2016-2017
processing season 2016-2017 stat playingtime
processing season 2014-2015
processing season 2014-2015 stat playingtime
processing season 2017-2018
processing season 2017-2018 stat playingtime
processing season 2018-2019
processing season 2018-2019 stat playingtime
processing season 2021-2022
processing season 2021-

In [ ]:
get_squad_data('2008-2009', 'stats')

In [ ]:
for i,(stat_type, table_id) in enumerate(stat_types):
  print("processing season {} stat {}".format('2014-2015', stat_type))
  get_squad_data('2014-2015', stat_type)

processing season 2014-2015 stat stats
processing season 2014-2015 stat keepers
processing season 2014-2015 stat shooting


In [ ]:
ranges = [(n, n+1) for n in range(2006, 2024)]

seasons = ["{}-{}".format(year1,year2) for (year1, year2) in ranges]
def get_fixtures_socres(season):
# Define the URL to scrape
  url = "https://fbref.com/en/comps/9/{}/schedule/{}-Premier-League-Scores-and-Fixtures".format(season, season)
  # Fetch the HTML content
  response = requests.get(url)
  html_content = response.content

  # Parse the HTML content
  soup = BeautifulSoup(html_content, "html.parser")

  # Extract the first table
  table = soup.find('table', {'class': 'stats_table'})
  if table == None:
    print("No table found")
    return
  # Create a Pandas DataFrame
  df = pd.read_html(str(table))[0]

  filename = '/content/drive/MyDrive/APS360_Project/Data_2/{}.xlsx'.format(season)
  # write to excel
  with pd.ExcelWriter(filename, mode='a', if_sheet_exists='new', engine='openpyxl') as writer:
          df.to_excel(writer, sheet_name='Fixture data')


def process_season(season):
    get_fixtures_socres(season)

num_processes = multiprocessing.cpu_count()
pool = multiprocessing.Pool(processes=num_processes)
pool.map(process_season, seasons)
pool.close()
pool.join()

In [ ]:
def get_squad_data(season, stat_type,league):
# Define the URL to scrape
  url = "https://fbref.com/en/comps/9/{}/{}/{}-{}".format(season,stat_type, season, league)

  # Fetch the HTML content
  response = requests.get(url)
  html_content = response.content

  # Parse the HTML content
  soup = BeautifulSoup(html_content, "html.parser")

  # Extract the first two tables
  tables = soup.find_all('table', {'class': 'stats_table'})[:2]
  if tables == None:
    print("No table found")
    return
  # Create a Pandas DataFrame for each table
  dfs = [pd.read_html(str(table))[0] for table in tables]
  filename = '/content/gdrive/MyDrive/APS360_Project/Data/EFL_{}.xlsx'.format(season)
  # write to excel
  if not os.path.isfile(filename):
    # Create a new file
    with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
        dfs[0].to_excel(writer, sheet_name='Squad Data {}'.format(stat_type))
  else:
    with pd.ExcelWriter(filename, mode='a', if_sheet_exists='new', engine='openpyxl') as writer:
      dfs[0].to_excel(writer, sheet_name='Squad Data {}'.format(stat_type))

def process_season(season):
  for i,(stat_type, table_id) in enumerate(stat_types):
    get_squad_data(season, stat_type,'Championship-Stats')

num_processes = multiprocessing.cpu_count()
pool = multiprocessing.Pool(processes=num_processes)
pool.map(process_season, seasons)
pool.close()
pool.join()